## Imports

In [1]:
import os

from pydantic import BaseModel, Field

from multi_agent_llm import AIOT, OpenAILLM

# Tool Runner

In [2]:
import subprocess
import sys
import tempfile
import shutil


def tool_runner(code: str, pip_dependencies: list) -> str:

    stdout = ""
    stderr = ""
    error = ""

    # Create a temporary directory for the virtual environment
    temp_dir = tempfile.mkdtemp()
    venv_dir = os.path.join(temp_dir, "venv")

    try:
        # Create the virtual environment
        subprocess.run([sys.executable, "-m", "venv", venv_dir], check=True)

        # Activate the virtual environment
        venv_python = os.path.join(venv_dir, "bin", "python")

        # Install dependencies
        if pip_dependencies:
            subprocess.run(
                [venv_python, "-m", "pip", "install", "--upgrade", "pip"],
                check=True
            )
            subprocess.run(
                [venv_python, "-m", "pip", "install"] + pip_dependencies,
                check=True
            )

        # Run the script
        process = subprocess.run(
            [venv_python, "-c", code],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True
        )
        stdout = process.stdout
        stderr = process.stderr

    except subprocess.CalledProcessError as e:
        error = f"Subprocess error: {str(e)}"
    except Exception as e:
        error = f"General error: {str(e)}"
    finally:
        # Clean up the virtual environment
        shutil.rmtree(temp_dir, ignore_errors=True)

    return "\n".join([stdout, stderr, error]).strip()

## Set OpenAI API Key

In [3]:
os.environ['OPENAI_API_KEY'] = open('../openai_token.key').read().strip()

## Initialize an LLM client

In [4]:
llm = OpenAILLM(model_name="gpt-4o-mini")

## Define an output schema

In [5]:
class QueryAnswer(BaseModel):
    explanation: str = Field(description="Explanation of the answer")
    answer: str = Field(description="Final Answer")

# ATTEMPT #1 -- tool to get weather data

**current outcome**: IDA tries over and over to correctly generate a working tool but requests to external APIs keep failing, resulting in a loop that is eventually terminated manually (no hard exit is implemented yet).

NOTE: bulb (💡) printouts are temporary for debugging

In [6]:
aiot = AIOT(
    llm=llm,
    iterations=2,
    answer_schema=QueryAnswer,
    tool_runner=tool_runner,
)

PROMPT = "Which city in Ontario has had the most rainfall this year? (2024)"

result = aiot.run(PROMPT)

result

💡 Received tool request from the brain. {'name': 'Real-Time Weather Data Fetcher', 'description': 'A tool that retrieves real-time weather data specifically focusing on rainfall amounts for major cities in Ontario for the year 2024. It should include options to filter the data based on specific cities or regions and specific time frames within the current year.', 'input': {'year': '2024', 'region': 'Ontario'}}
💡 Tool Generated:
 import requests
import json
from datetime import datetime

# Function to fetch real-time weather data

def get_weather_data(year, region):
    # Example weather API endpoint (You might need to replace it with a real one)
    api_key = 'your_api_key_here'
    url = f'https://api.weatherapi.com/v1/history.json?key={api_key}'
    cities = ['Toronto', 'Ottawa', 'Hamilton', 'Kitchener', 'London']
    rainfall_data = {}  # Dictionary to hold rainfall data for each city
    
    # Fetch data for each city
    for city in cities:
        try:
            response = req

KeyboardInterrupt: 

💡 Tool Generated:
 import requests
import json

def fetch_rainfall_data(year, region):
    url = 'https://api.weather.gc.ca/some_endpoint'  # Replace with actual API endpoint
    params = {
        'year': year,
        'region': region
    }
    response = requests.get(url, params=params)

    if response.status_code == 200:
        return response.json()
    else:
        return {'error': 'Failed to fetch data', 'status_code': response.status_code}

if __name__ == '__main__':
    year = 2024
    region = 'Ontario'
    rainfall_data = fetch_rainfall_data(year, region)
    print(json.dumps(rainfall_data))
  Using cached pip-24.3.1-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 23.0.1
    Uninstalling pip-23.0.1:
      Successfully uninstalled pip-23.0.1
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached charset_normalizer-3.4.0-cp310-cp310-macosx_11_0_arm64.whl.metadata (34 kB)
  Using cached idna-3.10-py3-none-a

# Attempt #2 -- tool that does calculations

**current outcome**: IDA prefers to solve and check solutions without initiating tool generation, despite rather explicit instruction to do so in prompt below and in system prompt as written in `iteration_of_thought.py`.

In [8]:
aiot = AIOT(
    llm=llm,
    iterations=2,
    answer_schema=QueryAnswer,
    tool_runner=tool_runner,
)

PROMPT = "Find and verify solutions for the roots of the function \
f(x) = 3x^5 - 7x^4 + 2x^3 - 5x^2 + 11sqrt(x) - 6/x + 2"

result = aiot.run(PROMPT)

result

<style>
            .discussion-result {
                font-family: Arial, sans-serif;
                max-width: 800px;
                margin: 20px auto;
                padding: 20px;
                border: 1px solid #ddd;
                border-radius: 10px;
                background-color: #f8f9fa;
                color: #333;
            }
            .query {
                background-color: #e9ecef;
                padding: 15px;
                border-radius: 8px;
                margin-bottom: 20px;
                font-weight: bold;
            }
            .thought-process {
                margin-left: 20px;
            }
            .turn {
                background-color: #ffffff;
                padding: 15px;
                margin-bottom: 15px;
                border-radius: 8px;
                box-shadow: 0 2px 4px rgba(0,0,0,0.1);
            }
            .turn.final {
                background-color: #d4edda;
            }
            .turn-content {
                margin-left: 20px;
            }
            .label {
                font-weight: bold;
                color: #495057;
                margin-right: 10px;
            }
            .ida-label {
                color: #0056b3;
            }
            .llma-label {
                color: #28a745;
            }
            pre {
                white-space: pre-wrap;
                word-wrap: break-word;
                background-color: #f1f3f5;
                padding: 10px;
                border-radius: 5px;
                max-height: 300px;
                overflow-y: auto;
            }
            .final-answer {
                background-color: #d4edda;
                padding: 15px;
                border-radius: 8px;
                margin-top: 20px;
            }
        </style>
        <div class='discussion-result'><h2>Discussion Result</h2><div class='query'>Find and verify solutions for the roots of the function f(x) = 3x^5 - 7x^4 + 2x^3 - 5x^2 + 11sqrt(x) - 6/x + 2</div><h3>Thought Process:</h3><div class='thought-process'>
                <div class='turn'>
                    <strong>Iteration 1:</strong>
                    <div class='turn-content'>
                        <p><span class='label ida-label'>IDA:</span> To find and verify the roots of the given function f(x) = 3x^5 - 7x^4 + 2x^3 - 5x^2 + 11sqrt(x) - 6/x + 2, the first step is to understand the function&#x27;s behavior. Since this polynomial includes a square root and a rational term, consider the domains and potential approaches for finding roots such as numerical methods like the Newton-Raphson method or using graphing to locate approximate roots. After identifying potential roots, use substitution or synthetic division to confirm them. Let&#x27;s start by analyzing the function and determining suitable methods to find its roots.</p>
                        <p><span class='label llma-label'>LLMA:</span> <pre>{
  explanation: To find roots of the function f(x) = 3x^5 - 7x^4 + 2x^3 - 5x^2 + 11√x - 6/x + 2, we first need to identify the domain. The function is only defined for x &gt; 0 due to the square root and the term -6/x. For initial analysis, we can graph the function to identify approximate roots. Based on numerical methods, like the Newton-Raphson method, we can refine these estimates. However, analytical methods for complex polynomials may involve substitution or special techniques to simplify.
  answer: 
}</pre></p>
                    </div>
                </div>
            
                <div class='turn'>
                    <strong>Iteration 2:</strong>
                    <div class='turn-content'>
                        <p><span class='label ida-label'>IDA:</span> Now that we have established that the function f(x) = 3x^5 - 7x^4 + 2x^3 - 5x^2 + 11sqrt(x) - 6/x + 2 is defined for x &gt; 0, let&#x27;s consider how we can numerically explore the roots further. It’s important to choose a suitable numerical method after observing th

# Attempt #3 -- tool performs google searches

**current outcome**: kinda paradoxically, the IDA actually tells the LLMA that direct access to google search is not possible in iteration 2 -- it then proceeds to answer the question without this, never requesting any tools.

In [6]:
aiot = AIOT(
    llm=llm,
    iterations=2,
    answer_schema=QueryAnswer,
    tool_runner=tool_runner,
)

PROMPT = "How many of the top 5 google search results for 'things to do in mexico' \
are related to food?"

result = aiot.run(PROMPT)

result

<style>
            .discussion-result {
                font-family: Arial, sans-serif;
                max-width: 800px;
                margin: 20px auto;
                padding: 20px;
                border: 1px solid #ddd;
                border-radius: 10px;
                background-color: #f8f9fa;
                color: #333;
            }
            .query {
                background-color: #e9ecef;
                padding: 15px;
                border-radius: 8px;
                margin-bottom: 20px;
                font-weight: bold;
            }
            .thought-process {
                margin-left: 20px;
            }
            .turn {
                background-color: #ffffff;
                padding: 15px;
                margin-bottom: 15px;
                border-radius: 8px;
                box-shadow: 0 2px 4px rgba(0,0,0,0.1);
            }
            .turn.final {
                background-color: #d4edda;
            }
            .turn-content {
                margin-left: 20px;
            }
            .label {
                font-weight: bold;
                color: #495057;
                margin-right: 10px;
            }
            .ida-label {
                color: #0056b3;
            }
            .llma-label {
                color: #28a745;
            }
            pre {
                white-space: pre-wrap;
                word-wrap: break-word;
                background-color: #f1f3f5;
                padding: 10px;
                border-radius: 5px;
                max-height: 300px;
                overflow-y: auto;
            }
            .final-answer {
                background-color: #d4edda;
                padding: 15px;
                border-radius: 8px;
                margin-top: 20px;
            }
        </style>
        <div class='discussion-result'><h2>Discussion Result</h2><div class='query'>How many of the top 5 google search results for &#x27;things to do in mexico&#x27; are related to food?</div><h3>Thought Process:</h3><div class='thought-process'>
                <div class='turn'>
                    <strong>Iteration 1:</strong>
                    <div class='turn-content'>
                        <p><span class='label ida-label'>IDA:</span> To answer the query, you first need to conceptualize what types of search results come up when querying &#x27;things to do in Mexico.&#x27; Consider common categories present in these results and whether food falls into these categories. What do you already know about search results related to this topic? Do you have any strategies to find or assess the current top search results? Think about how you could analyze the content of these results easily. You might have to think about using a search engine to check the current top results accurately, but let&#x27;s first clarify what those top results typically look like.</p>
                        <p><span class='label llma-label'>LLMA:</span> <pre>{
  explanation: Typically, when searching for &#x27;things to do in Mexico,&#x27; the top search results are a mix of activities, attractions, and cultural experiences. These results often include categories such as outdoor activities, historical sites, beaches, festivals, and, importantly, local cuisine. Since food is a significant part of many travel experiences, it is likely that at least some of the top search results contain food-related recommendations, such as popular dishes to try, best restaurants, or culinary tours.
  answer: To determine the exact number of food-related results in the current top 5 search results, an actual search must be conducted.
}</pre></p>
                    </div>
                </div>
            
                <div class='turn'>
                    <strong>Iteration 2:</strong>
                    <div class='turn-content'>
                        <p><span class='label ida-label'>IDA:</span> Let&#x27;s build on your understanding of search results. You mentioned that local